In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

from code_types import CodeFormatter

In [ ]:
from e2b_code_interpreter import Sandbox
import os

# Create the sandbox
cwd = "/home/user/code"
sandbox = Sandbox(api_key=os.getenv("E2B_API_KEY"), cwd=cwd)




In [ ]:
code_formatter = CodeFormatter()

In [ ]:
local_dir = os.path.join(os.getcwd(), "..", "generated_code_20240805_061004")

In [ ]:
import mimetypes

def is_binary(file_path):
    mime_type, _ = mimetypes.guess_type(file_path)
    return mime_type and not mime_type.startswith('text')

In [ ]:
for root, dirs, files in os.walk(local_dir):
    for file in files:
        local_path = os.path.join(root, file)
        relative_path = os.path.relpath(local_path, local_dir)
        sandbox_path = f"{cwd}/{relative_path}"
        
        # Create directories if they don't exist
        sandbox_dir = os.path.dirname(sandbox_path)
        sandbox.filesystem.make_dir(sandbox_dir, timeout=None)
        
        if is_binary(local_path):
            with open(local_path, 'rb') as file_obj:
                content = file_obj.read()
            sandbox.filesystem.write_bytes(sandbox_path, content, timeout=None)
        else:
            # Directly read and write text files
            with open(local_path, 'r', encoding='utf-8') as file_obj:
                content = file_obj.read()
            content = content.replace("\\'", "'").replace('\\"', '"')
            sandbox.filesystem.write(sandbox_path, content, timeout=None)


        print(f"Uploaded: {local_path} -> {sandbox_path}")

In [ ]:
sandbox.filesystem.list("/home/user/code")

In [ ]:
print("Installing requirements...")
install_result = sandbox.process.start_and_wait(f"pip install -r {cwd}/requirements.txt", timeout=300)
print("Pip install stdout:", install_result.stdout)
print("Pip install stderr:", install_result.stderr)

In [ ]:
# print("Installing requirements...")
# format_and_lint_result = sandbox.process.start_and_wait(f"python {cwd}/format_and_lint.py", timeout=300)
# print("Format and lint stdout:", format_and_lint_result.stdout)
# print("Format and lint stderr:", format_and_lint_result.stderr)

In [ ]:
print("\nRunning tests...")
test_result = sandbox.process.start_and_wait(f"python {cwd}/run_tests.py", timeout=300)
print("Test stdout:", test_result.stdout)
print("Test stderr:", test_result.stderr)

In [ ]:
# sandbox.close()